# ECVL Dataset YAML visualization

#### This notebook is for visualizing the images in each split of an ECVL Dataset YAML file

## Imports

In [ ]:
import os
import random

import yaml
import ipywidgets as widgets
from IPython.display import display

from lib.plot_utils import show_images
from lib.image_processing import load_numpy_data

## Config

In [ ]:
# Path to the yaml to inspect
preproc_data_path = "../../../datasets/preproc_lungsregion_BIMCV-COVID19/"
yaml_path = os.path.join(preproc_data_path, "ecvl_bimcv_covid19.yaml")

# Split to take the images from
#target_split = "training"
#target_split = "validation"
target_split = "test"

## Select the set of images to visualize

In [ ]:
# Read the YAML data
with open(yaml_path, "r") as stream:
    try:
        yaml_data = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# Get the full path. "relative_paths" are relative to the main data folder
#images_data = [(sub, sess, os.path.join(subjects_path, path)) for sub, sess, path in images_data]
split_indexes = yaml_data['split'][target_split]
split_samples = [yaml_data['images'][idx] for idx in split_indexes]

# Prepare the list structure to store the samples
images_data = []
for sample in split_samples:
    age = sample['values']['age']
    gender = sample['values']['gender']
    path = os.path.join(preproc_data_path, sample['location'])
    label = sample['label'][0]
    images_data.append((age, gender, path, label))

In [ ]:
if len(images_data) == 0:
    raise Exception("There are no images to show!")
    
current_image = 0  # To track the index of the current image to show

AGE, GENDER, PATH, LABEL = 0, 1, 2, 3  # Auxiliay indexes

# Create output widgets to control the layout
output = widgets.Output()  # To show the images
log_out =  widgets.Output()  # To show text data

def show_image(index):
    """Shows the image selected by the index provided"""
    output.clear_output()
    with output:
        show_images([load_numpy_data(images_data[index][PATH])])

def show_data(index):
    log_out.clear_output()
    with log_out:
        print(f"Image {index+1}/{len(images_data)}")
        age = images_data[index][AGE]
        gender = images_data[index][GENDER]
        label = images_data[index][LABEL]
        print(f"{age=}\n{gender=}\n{label=}")
        
show_image(current_image)  # Show the first image
show_data(current_image)   # Log sample data

# Create the button to skip the images
buttons_layout = widgets.Layout(width='100px', height='100px')
b_next = widgets.Button(description="Next", button_style="info", layout=buttons_layout, icon='arrow-right')

# Prepare the function callbacks for the buttons to label the current image

def next_image(label=None):
    """
    Function to load and show the next image after pressing the next button.
    This function also controls the logs.
    """
    # Bring the global variable to the scope
    global current_image   
    # Pass to the next image
    current_image += 1
    
    # Show the next image
    if current_image < len(images_data):
        show_image(current_image)
        show_data(current_image)
    else:
        b_next.disabled = True
        with log_out:
            print("\nYOU REACHED THE END OF THE SAMPLES LIST!")
        
# Assign the button callback
b_next.on_click(next_image)
    
# Configure the layout of the elements and show it
buttons = widgets.VBox([b_next])
display(widgets.HBox([output, buttons, log_out]))